# Deep Convolutional GANs

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from G import G as G
from D import D as D

In [2]:
batchSize = 64 
imageSize = 64 

transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) 

dataset = dset.CIFAR10(root = './data', download = True, transform = transform) 
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) 

/home/hvyd/anaconda3/lib/python3.5/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Files already downloaded and verified


In [3]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# Generator

In [4]:
netG = G()
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

# Discriminator


In [5]:
netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

# Training the DCGANs

In [6]:
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

for epoch in range(25):

    for i, data in enumerate(dataloader, 0):
        
        netD.zero_grad()
        
        # Training the discriminator with a real image of the dataset
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output, target)
        
        # Backpropagating the total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step() 
        
        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.data[0], errG.data[0]))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)
        

/home/hvyd/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:38: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[0/25][0/782] Loss_D: 1.8682 Loss_G: 4.5517
[0/25][1/782] Loss_D: 1.3389 Loss_G: 6.0139
[0/25][2/782] Loss_D: 0.8093 Loss_G: 5.5869
[0/25][3/782] Loss_D: 0.9610 Loss_G: 6.1590
[0/25][4/782] Loss_D: 0.7666 Loss_G: 6.6839
[0/25][5/782] Loss_D: 1.0948 Loss_G: 5.8599
[0/25][6/782] Loss_D: 1.2627 Loss_G: 8.2935
[0/25][7/782] Loss_D: 0.6870 Loss_G: 7.9181
[0/25][8/782] Loss_D: 0.6718 Loss_G: 7.8171
[0/25][9/782] Loss_D: 0.6594 Loss_G: 9.2848
[0/25][10/782] Loss_D: 0.8783 Loss_G: 8.6805
[0/25][11/782] Loss_D: 0.9751 Loss_G: 10.2034
[0/25][12/782] Loss_D: 0.6777 Loss_G: 7.8494
[0/25][13/782] Loss_D: 1.1714 Loss_G: 12.6030
[0/25][14/782] Loss_D: 0.3321 Loss_G: 9.7251
[0/25][15/782] Loss_D: 0.7122 Loss_G: 10.3030
[0/25][16/782] Loss_D: 0.5205 Loss_G: 10.7529
[0/25][17/782] Loss_D: 0.3464 Loss_G: 9.9848
[0/25][18/782] Loss_D: 0.6510 Loss_G: 12.6782
[0/25][19/782] Loss_D: 0.2297 Loss_G: 9.3830
[0/25][20/782] Loss_D: 0.6528 Loss_G: 14.4323
[0/25][21/782] Loss_D: 0.1505 Loss_G: 12.4194
[0/25][22/782

[0/25][180/782] Loss_D: 0.2561 Loss_G: 4.9087
[0/25][181/782] Loss_D: 0.3121 Loss_G: 4.4821
[0/25][182/782] Loss_D: 0.4487 Loss_G: 5.7360
[0/25][183/782] Loss_D: 0.2215 Loss_G: 5.2288
[0/25][184/782] Loss_D: 0.2085 Loss_G: 4.8589
[0/25][185/782] Loss_D: 0.3274 Loss_G: 4.5996
[0/25][186/782] Loss_D: 0.5355 Loss_G: 8.4879
[0/25][187/782] Loss_D: 0.2839 Loss_G: 7.1639
[0/25][188/782] Loss_D: 0.1226 Loss_G: 4.6807
[0/25][189/782] Loss_D: 0.3505 Loss_G: 8.5466
[0/25][190/782] Loss_D: 0.4403 Loss_G: 5.8916
[0/25][191/782] Loss_D: 0.1758 Loss_G: 5.6044
[0/25][192/782] Loss_D: 0.2526 Loss_G: 7.1248
[0/25][193/782] Loss_D: 0.1720 Loss_G: 6.1010
[0/25][194/782] Loss_D: 0.1929 Loss_G: 5.9900
[0/25][195/782] Loss_D: 0.4476 Loss_G: 6.4023
[0/25][196/782] Loss_D: 0.2796 Loss_G: 5.6890
[0/25][197/782] Loss_D: 0.1510 Loss_G: 7.5480
[0/25][198/782] Loss_D: 0.2137 Loss_G: 5.7095
[0/25][199/782] Loss_D: 0.1728 Loss_G: 7.4634
[0/25][200/782] Loss_D: 0.1580 Loss_G: 6.2937
[0/25][201/782] Loss_D: 0.2066 Los

[0/25][358/782] Loss_D: 0.1476 Loss_G: 5.6503
[0/25][359/782] Loss_D: 0.0757 Loss_G: 6.1488
[0/25][360/782] Loss_D: 0.1568 Loss_G: 5.3165
[0/25][361/782] Loss_D: 0.0920 Loss_G: 5.2855
[0/25][362/782] Loss_D: 0.1487 Loss_G: 5.9162
[0/25][363/782] Loss_D: 0.0921 Loss_G: 5.8038
[0/25][364/782] Loss_D: 0.1671 Loss_G: 5.8410
[0/25][365/782] Loss_D: 0.1064 Loss_G: 6.3320
[0/25][366/782] Loss_D: 0.1478 Loss_G: 5.7936
[0/25][367/782] Loss_D: 0.1375 Loss_G: 6.9008
[0/25][368/782] Loss_D: 0.0545 Loss_G: 6.7630
[0/25][369/782] Loss_D: 0.1179 Loss_G: 6.0058
[0/25][370/782] Loss_D: 0.1235 Loss_G: 5.3580
[0/25][371/782] Loss_D: 0.2148 Loss_G: 6.1477
[0/25][372/782] Loss_D: 0.1089 Loss_G: 5.8225
[0/25][373/782] Loss_D: 0.0921 Loss_G: 5.6247
[0/25][374/782] Loss_D: 0.0891 Loss_G: 5.8649
[0/25][375/782] Loss_D: 0.0853 Loss_G: 5.5349
[0/25][376/782] Loss_D: 0.0926 Loss_G: 5.0685
[0/25][377/782] Loss_D: 0.0718 Loss_G: 6.1748
[0/25][378/782] Loss_D: 0.1214 Loss_G: 5.3121
[0/25][379/782] Loss_D: 0.0740 Los

[0/25][536/782] Loss_D: 0.3646 Loss_G: 9.5599
[0/25][537/782] Loss_D: 0.3914 Loss_G: 7.8024
[0/25][538/782] Loss_D: 0.2609 Loss_G: 5.2251
[0/25][539/782] Loss_D: 0.6158 Loss_G: 9.5913
[0/25][540/782] Loss_D: 1.7468 Loss_G: 3.2037
[0/25][541/782] Loss_D: 1.4679 Loss_G: 11.7525
[0/25][542/782] Loss_D: 0.4254 Loss_G: 12.7723
[0/25][543/782] Loss_D: 0.3548 Loss_G: 9.3538
[0/25][544/782] Loss_D: 0.1547 Loss_G: 6.2425
[0/25][545/782] Loss_D: 0.3235 Loss_G: 5.7829
[0/25][546/782] Loss_D: 0.2720 Loss_G: 5.1331
[0/25][547/782] Loss_D: 0.2081 Loss_G: 4.9564
[0/25][548/782] Loss_D: 0.5209 Loss_G: 6.7756
[0/25][549/782] Loss_D: 0.3490 Loss_G: 5.2879
[0/25][550/782] Loss_D: 0.2683 Loss_G: 3.6114
[0/25][551/782] Loss_D: 0.5350 Loss_G: 6.8188
[0/25][552/782] Loss_D: 0.4884 Loss_G: 5.2832
[0/25][553/782] Loss_D: 0.1724 Loss_G: 4.2460
[0/25][554/782] Loss_D: 0.2677 Loss_G: 6.2087
[0/25][555/782] Loss_D: 0.1780 Loss_G: 5.4748
[0/25][556/782] Loss_D: 0.1829 Loss_G: 3.7575
[0/25][557/782] Loss_D: 0.2278 L

...

[23/25][772/782] Loss_D: 0.8682 Loss_G: 2.5771
[23/25][773/782] Loss_D: 0.5847 Loss_G: 1.5787
[23/25][774/782] Loss_D: 0.6199 Loss_G: 3.6442
[23/25][775/782] Loss_D: 0.8746 Loss_G: 1.7603
[23/25][776/782] Loss_D: 0.8826 Loss_G: 2.6601
[23/25][777/782] Loss_D: 0.7020 Loss_G: 2.6817
[23/25][778/782] Loss_D: 0.7981 Loss_G: 1.3554
[23/25][779/782] Loss_D: 0.7732 Loss_G: 2.6712
[23/25][780/782] Loss_D: 0.6595 Loss_G: 3.0204
[23/25][781/782] Loss_D: 0.7713 Loss_G: 1.3172
[24/25][0/782] Loss_D: 1.0139 Loss_G: 4.2619
[24/25][1/782] Loss_D: 0.6016 Loss_G: 3.0975
[24/25][2/782] Loss_D: 0.7335 Loss_G: 1.1689
[24/25][3/782] Loss_D: 0.8745 Loss_G: 3.6636
[24/25][4/782] Loss_D: 0.2971 Loss_G: 3.8695
[24/25][5/782] Loss_D: 0.5174 Loss_G: 1.8745
[24/25][6/782] Loss_D: 0.8617 Loss_G: 3.6400
[24/25][7/782] Loss_D: 0.5437 Loss_G: 2.5474
[24/25][8/782] Loss_D: 0.4712 Loss_G: 2.4647
[24/25][9/782] Loss_D: 0.5205 Loss_G: 3.0002
[24/25][10/782] Loss_D: 0.5852 Loss_G: 3.0012
[24/25][11/782] Loss_D: 0.7400 Los

[24/25][167/782] Loss_D: 0.3880 Loss_G: 2.0351
[24/25][168/782] Loss_D: 0.6592 Loss_G: 3.4254
[24/25][169/782] Loss_D: 0.4547 Loss_G: 2.5797
[24/25][170/782] Loss_D: 0.4771 Loss_G: 2.3888
[24/25][171/782] Loss_D: 0.4672 Loss_G: 3.1786
[24/25][172/782] Loss_D: 0.4096 Loss_G: 2.9686
[24/25][173/782] Loss_D: 0.4018 Loss_G: 2.0429
[24/25][174/782] Loss_D: 0.2620 Loss_G: 2.9313
[24/25][175/782] Loss_D: 0.4285 Loss_G: 3.3957
[24/25][176/782] Loss_D: 0.6695 Loss_G: 1.4425
[24/25][177/782] Loss_D: 0.5155 Loss_G: 3.7501
[24/25][178/782] Loss_D: 0.5294 Loss_G: 2.1749
[24/25][179/782] Loss_D: 0.6176 Loss_G: 3.1412
[24/25][180/782] Loss_D: 0.6474 Loss_G: 1.5363
[24/25][181/782] Loss_D: 0.6426 Loss_G: 2.9318
[24/25][182/782] Loss_D: 0.3003 Loss_G: 3.4332
[24/25][183/782] Loss_D: 0.6188 Loss_G: 1.5347
[24/25][184/782] Loss_D: 0.6993 Loss_G: 3.3783
[24/25][185/782] Loss_D: 0.5734 Loss_G: 2.5420
[24/25][186/782] Loss_D: 0.4617 Loss_G: 2.0218
[24/25][187/782] Loss_D: 0.7524 Loss_G: 3.2028
[24/25][188/7

[24/25][342/782] Loss_D: 0.4709 Loss_G: 3.9520
[24/25][343/782] Loss_D: 0.5050 Loss_G: 2.8743
[24/25][344/782] Loss_D: 0.3616 Loss_G: 2.5417
[24/25][345/782] Loss_D: 0.7108 Loss_G: 3.8408
[24/25][346/782] Loss_D: 0.6463 Loss_G: 1.8196
[24/25][347/782] Loss_D: 0.6337 Loss_G: 3.4733
[24/25][348/782] Loss_D: 0.4430 Loss_G: 3.3413
[24/25][349/782] Loss_D: 0.6727 Loss_G: 1.4586
[24/25][350/782] Loss_D: 0.5598 Loss_G: 3.9343
[24/25][351/782] Loss_D: 0.7290 Loss_G: 1.7963
[24/25][352/782] Loss_D: 0.6412 Loss_G: 3.3285
[24/25][353/782] Loss_D: 0.3604 Loss_G: 4.4750
[24/25][354/782] Loss_D: 0.5383 Loss_G: 2.5992
[24/25][355/782] Loss_D: 0.2439 Loss_G: 2.7130
[24/25][356/782] Loss_D: 0.5629 Loss_G: 4.1234
[24/25][357/782] Loss_D: 0.6588 Loss_G: 1.6878
[24/25][358/782] Loss_D: 0.4547 Loss_G: 4.5789
[24/25][359/782] Loss_D: 0.1003 Loss_G: 4.9159
[24/25][360/782] Loss_D: 0.2636 Loss_G: 3.5439
[24/25][361/782] Loss_D: 0.1061 Loss_G: 3.2302
[24/25][362/782] Loss_D: 0.1920 Loss_G: 4.3616
[24/25][363/7

[24/25][517/782] Loss_D: 0.0087 Loss_G: 5.6884
[24/25][518/782] Loss_D: 0.0028 Loss_G: 7.1011
[24/25][519/782] Loss_D: 0.0075 Loss_G: 5.9879
[24/25][520/782] Loss_D: 0.0294 Loss_G: 5.2582
[24/25][521/782] Loss_D: 0.0076 Loss_G: 6.9595
[24/25][522/782] Loss_D: 0.0209 Loss_G: 5.2717
[24/25][523/782] Loss_D: 0.0027 Loss_G: 7.6359
[24/25][524/782] Loss_D: 0.0091 Loss_G: 5.9317
[24/25][525/782] Loss_D: 0.0021 Loss_G: 8.6549
[24/25][526/782] Loss_D: 0.0031 Loss_G: 7.3141
[24/25][527/782] Loss_D: 0.0189 Loss_G: 5.9602
[24/25][528/782] Loss_D: 0.0187 Loss_G: 5.6059
[24/25][529/782] Loss_D: 0.0087 Loss_G: 7.5474
[24/25][530/782] Loss_D: 0.0265 Loss_G: 5.2843
[24/25][531/782] Loss_D: 0.0022 Loss_G: 8.2433
[24/25][532/782] Loss_D: 0.0132 Loss_G: 5.9263
[24/25][533/782] Loss_D: 0.0022 Loss_G: 8.0584
[24/25][534/782] Loss_D: 0.0087 Loss_G: 6.9193
[24/25][535/782] Loss_D: 0.0231 Loss_G: 5.3311
[24/25][536/782] Loss_D: 0.0072 Loss_G: 8.3901
[24/25][537/782] Loss_D: 0.0121 Loss_G: 5.4678
[24/25][538/7

[24/25][692/782] Loss_D: 0.2348 Loss_G: 3.9090
[24/25][693/782] Loss_D: 0.2660 Loss_G: 3.5262
[24/25][694/782] Loss_D: 0.5750 Loss_G: 1.7593
[24/25][695/782] Loss_D: 0.4128 Loss_G: 3.7408
[24/25][696/782] Loss_D: 0.1787 Loss_G: 4.6869
[24/25][697/782] Loss_D: 0.3142 Loss_G: 2.5855
[24/25][698/782] Loss_D: 0.1701 Loss_G: 3.4847
[24/25][699/782] Loss_D: 0.2582 Loss_G: 3.9693
[24/25][700/782] Loss_D: 0.2056 Loss_G: 3.3441
[24/25][701/782] Loss_D: 0.2347 Loss_G: 4.1091
[24/25][702/782] Loss_D: 0.1646 Loss_G: 3.7734
[24/25][703/782] Loss_D: 0.2272 Loss_G: 2.5945
[24/25][704/782] Loss_D: 0.2434 Loss_G: 3.7770
[24/25][705/782] Loss_D: 0.1521 Loss_G: 4.2867
[24/25][706/782] Loss_D: 0.2498 Loss_G: 3.1911
[24/25][707/782] Loss_D: 0.1841 Loss_G: 3.2865
[24/25][708/782] Loss_D: 0.1001 Loss_G: 4.7718
[24/25][709/782] Loss_D: 0.1243 Loss_G: 4.1865
[24/25][710/782] Loss_D: 0.1618 Loss_G: 3.6419
[24/25][711/782] Loss_D: 0.0565 Loss_G: 4.4987
[24/25][712/782] Loss_D: 0.1875 Loss_G: 5.2616
[24/25][713/7